In [1]:
# Marketing Cost Prediction - Gradient Boosting Model
# This notebook implements a comprehensive gradient boosting model with feature engineering,
# proper scaling, hyperparameter tuning, and model interpretability

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score, 
    mean_absolute_percentage_error, explained_variance_score
)
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(314)

# Configuration
DATA_PATH = 'Data_for_taining_14072025.csv'
TARGET = 'marketing_cost'
RANDOM_STATE = 314

print("=== Marketing Cost Prediction - Gradient Boosting Model ===\n")

# Load data
df = pd.read_csv(DATA_PATH)

# Display first few rows of the dataset
print("First 5 rows of the dataset:")
print(df.head())

=== Marketing Cost Prediction - Gradient Boosting Model ===

First 5 rows of the dataset:
   Unnamed: 0     aov_eur  available_stock_value_after_discount_complete_eur  \
0  2022-03-14  144.754190                                       4.834682e+07   
1  2022-03-21  149.213739                                       4.980875e+07   
2  2022-03-28  142.841626                                       5.277616e+07   
3  2022-04-04  151.042129                                       5.601095e+07   
4  2022-04-11  153.828542                                       5.911150e+07   

   avg_temp       cpc  cr_tracked_%  email_recipients  email_visits  \
0  4.771429  0.369265      0.030110          754257.0       23604.0   
1  6.742857  0.397933      0.030137          966739.0       43888.0   
2  8.657143  0.391481      0.029470          580566.0       12686.0   
3  3.657143  0.404580      0.030292         1118797.0       38267.0   
4  6.900000  0.397859      0.026801          734504.0       20674.0   

  

In [2]:
# 1. Data Loading and Initial Exploration
print("1. Loading and exploring data...")

# Load data
df = pd.read_csv(DATA_PATH)
df.rename(columns={'Unnamed: 0': 'date'}, inplace=True)
df['date'] = pd.to_datetime(df['date'])
df.sort_values('date', inplace=True)
df.set_index('date', inplace=True)

# Remove first and last week as in original approach
df = df.iloc[1:-1]

print(f"Dataset shape: {df.shape}")
print(f"Date range: {df.index.min()} to {df.index.max()}")
print(f"Number of features: {len(df.columns) - 1}")  # Excluding target
print(f"Target variable: {TARGET}")

# Display basic statistics
print("\nTarget variable statistics:")
print(df[TARGET].describe())

# Check for missing values
missing_values = df.isnull().sum()
print(f"\nMissing values per column:")
print(missing_values[missing_values > 0])

1. Loading and exploring data...
Dataset shape: (171, 28)
Date range: 2022-03-21 00:00:00 to 2025-06-23 00:00:00
Number of features: 27
Target variable: marketing_cost

Target variable statistics:
count    1.710000e+02
mean     4.228278e+05
std      1.456366e+05
min      2.241357e+05
25%      3.334185e+05
50%      3.861772e+05
75%      4.873196e+05
max      1.251066e+06
Name: marketing_cost, dtype: float64

Missing values per column:
Series([], dtype: int64)


In [3]:
# 2. Feature Engineering
print("\n2. Feature Engineering...")

def create_lag_features(df, target_col, lags=[1, 2, 4]):
    """Create lag features for time series data"""
    df_lagged = df.copy()
    for lag in lags:
        df_lagged[f'{target_col}_lag_{lag}'] = df_lagged[target_col].shift(lag)
    return df_lagged

def create_rolling_features(df, target_col, windows=[2, 4]):
    """Create rolling window features"""
    df_rolling = df.copy()
    for window in windows:
        df_rolling[f'{target_col}_rolling_mean_{window}'] = df_rolling[target_col].rolling(window=window).mean()
        df_rolling[f'{target_col}_rolling_std_{window}'] = df_rolling[target_col].rolling(window=window).std()
    return df_rolling

def create_interaction_features(df):
    """Create interaction features between key variables"""
    df_interactions = df.copy()
    
    # Revenue-related interactions
    df_interactions['revenue_per_visit'] = df_interactions['number_orders'] * df_interactions['aov_eur'] / df_interactions['number_visits']
    df_interactions['conversion_rate'] = df_interactions['number_orders'] / df_interactions['number_visits']
    
    # Marketing efficiency
    df_interactions['roas'] = (df_interactions['number_orders'] * df_interactions['aov_eur']) / df_interactions['marketing_cost']
    
    return df_interactions

# Apply feature engineering
print("Creating lag features...")
df_engineered = create_lag_features(df, TARGET, lags=[1, 2, 4])

print("Creating rolling features...")
df_engineered = create_rolling_features(df_engineered, TARGET, windows=[2, 4])

print("Creating interaction features...")
df_engineered = create_interaction_features(df_engineered)

# Handle infinite values from division
df_engineered = df_engineered.replace([np.inf, -np.inf], np.nan)

print(f"Shape after feature engineering: {df_engineered.shape}")
print(f"New features created: {df_engineered.shape[1] - df.shape[1]}")


2. Feature Engineering...
Creating lag features...
Creating rolling features...
Creating interaction features...
Shape after feature engineering: (171, 38)
New features created: 10


In [4]:
# 3. Data Preprocessing
print("\n3. Data Preprocessing...")

# Drop rows with missing values (as per business requirement)
print(f"Before dropping missing values: {df_engineered.shape[0]} rows")
df_clean = df_engineered.dropna()
print(f"After dropping missing values: {df_clean.shape[0]} rows")

# Separate features and target
X = df_clean.drop(columns=[TARGET])
y = df_clean[TARGET]

print(f"Final dataset shape: {X.shape}")
print(f"Feature columns: {list(X.columns)}")

# Split data (keeping temporal order)
train_size = int(0.8 * len(X))
X_train = X.iloc[:train_size]
X_test = X.iloc[train_size:]
y_train = y.iloc[:train_size]
y_test = y.iloc[train_size:]

print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")


3. Data Preprocessing...
Before dropping missing values: 171 rows
After dropping missing values: 167 rows
Final dataset shape: (167, 37)
Feature columns: ['aov_eur', 'available_stock_value_after_discount_complete_eur', 'avg_temp', 'cpc', 'cr_tracked_%', 'email_recipients', 'email_visits', 'internalWeeks_until_SeasonalSaleStart', 'internal_Week_of_FW_Season', 'internal_Week_of_SS_Season', 'is_Peak_Driving_Public_Holiday_week', 'is_Sun_to_Mon_Shift_week', 'is_black_week_event', 'is_email_campaign_type_deal', 'is_email_campaign_type_liveshop', 'is_email_campaign_type_newsletter', 'is_percentage_on_top', 'is_percentage_on_top_applicable', 'is_season_sale_event', 'is_temp_drop_flag', 'number_days_after_last_event', 'number_days_till_next_event', 'number_orders', 'number_visits', 'sku_with_discount_%', 'stock_discount_rate_total_%', 'target_cpr', 'marketing_cost_lag_1', 'marketing_cost_lag_2', 'marketing_cost_lag_4', 'marketing_cost_rolling_mean_2', 'marketing_cost_rolling_std_2', 'marketin

In [5]:
# 4. Feature Scaling
print("\n4. Feature Scaling...")

# Initialize scaler
scaler = StandardScaler()

# Fit scaler on training data only
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame for easier handling
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print("Feature scaling completed")
print(f"Training data scaled shape: {X_train_scaled_df.shape}")
print(f"Test data scaled shape: {X_test_scaled_df.shape}")

# Show scaling effect on a few features
print("\nScaling effect on sample features:")
sample_features = ['aov_eur', 'available_stock_value_after_discount_complete_eur', 'avg_temp']
for feature in sample_features:
    if feature in X_train.columns:
        print(f"{feature}:")
        print(f"  Original - Mean: {X_train[feature].mean():.2f}, Std: {X_train[feature].std():.2f}")
        print(f"  Scaled - Mean: {X_train_scaled_df[feature].mean():.2f}, Std: {X_train_scaled_df[feature].std():.2f}")


4. Feature Scaling...
Feature scaling completed
Training data scaled shape: (133, 37)
Test data scaled shape: (34, 37)

Scaling effect on sample features:
aov_eur:
  Original - Mean: 157.94, Std: 12.97
  Scaled - Mean: 0.00, Std: 1.00
available_stock_value_after_discount_complete_eur:
  Original - Mean: 81141833.61, Std: 17128045.95
  Scaled - Mean: 0.00, Std: 1.00
avg_temp:
  Original - Mean: 11.93, Std: 6.41
  Scaled - Mean: 0.00, Std: 1.00


In [6]:
# 5. Hyperparameter Tuning
print("\n5. Hyperparameter Tuning...")

# Define parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 6, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'min_samples_split': [0.03, 0.05, 0.04],
    'subsample': [0.8, 0.9, 1.0]
}

# Initialize base model
base_model = GradientBoostingRegressor(random_state=RANDOM_STATE)

# Perform grid search
print("Performing grid search...")
grid_search = GridSearchCV(
    base_model,
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train_scaled_df, y_train)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best cross-validation score: {-grid_search.best_score_:.2f}")

# Get best model
best_model = grid_search.best_estimator_


5. Hyperparameter Tuning...
Performing grid search...
Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Best parameters: {'learning_rate': 0.2, 'max_depth': 6, 'min_samples_split': 0.03, 'n_estimators': 200, 'subsample': 0.8}
Best cross-validation score: 1970664557.25


In [7]:
# 6. Model Training and Evaluation
print("\n6. Model Training and Evaluation...")

# Train the best model
best_model.fit(X_train_scaled_df, y_train)

# Make predictions
y_train_pred = best_model.predict(X_train_scaled_df)
y_test_pred = best_model.predict(X_test_scaled_df)

# Calculate metrics
def calculate_metrics(y_true, y_pred, set_name):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    evs = explained_variance_score(y_true, y_pred)
    
    print(f"\n{set_name} Metrics:")
    print(f"MAE: {mae:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R²: {r2:.4f}")
    print(f"MAPE: {mape:.4f}")
    print(f"Explained Variance: {evs:.4f}")
    
    return {
        'mae': mae,
        'rmse': rmse,
        'r2': r2,
        'mape': mape,
        'explained_variance': evs
    }

# Calculate metrics for both sets
train_metrics = calculate_metrics(y_train, y_train_pred, "Training")
test_metrics = calculate_metrics(y_test, y_test_pred, "Test")


6. Model Training and Evaluation...

Training Metrics:
MAE: 0.00
RMSE: 0.00
R²: 1.0000
MAPE: 0.0000
Explained Variance: 1.0000

Test Metrics:
MAE: 25478.07
RMSE: 32053.35
R²: 0.9644
MAPE: 0.0487
Explained Variance: 0.9779
